# AIP 4 - Project Rechin
## Strategic Thinking - CA4 - Sprint 2
## Project Rechin - Can an AI Beat a Human Trader

***

## Model configuration: customisation posibilities

***

## LSTM Layer parameters and defaults

***

## Library imports

In [2]:
# Data acquisition
import pickle
import mysql.connector 
from sqlalchemy import create_engine
import os

# Data analysis basics
import numpy as np
import pandas as pd
import datetime

#EDA
#import pandas_profiling # https://anaconda.org/conda-forge/pandas-profiling # conda install ipywidgets

#Machine Learning
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard

# ML > LSTM 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Input

# ML > Save model
from tensorflow.keras.models import model_from_json

# ML > Optimizer > Adam just for trying
from tensorflow.keras.optimizers import Adam

# ML > Metrics
from tensorflow.keras import metrics as metrics_tf
import sklearn.metrics as metrics

# ML > GPU for tensorflow
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

# Feature Scaling
from sklearn.preprocessing import StandardScaler

#Viz
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
# Set plot size 
from pylab import rcParams
%matplotlib inline

#Cosmetic: Disabling notebook warnings
import warnings
warnings.filterwarnings("ignore")

# Plot LSTM Model
# https://keras.io/api/utils/model_plotting_utils/
from tensorflow.keras.utils import plot_model

In [3]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

## Check If GPU or CPU

GPU runs much faster for model training than CPU but you must install CUDA first. 

In [4]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
if gpu_devices:
    print('Using GPU')
    tf.config.experimental.set_memory_growth(gpu_devices[0], True)
else:
    print('Using CPU')

Using GPU


## Credentials To Login

In [5]:
api_key = "enterLater"
startDate = '2021-10-07'
missingSymbols = []

# Credentials to database connection
hostname="database-rechin.123456.eu-west-1.rds.amazonaws.com" #Antonio's AWS
uname="admin"
pwd="rechin"
dbName = 'rechindbcct'

# Connect to Specific Database
db = mysql.connector.connect(host= hostname,
  user=uname, password=pwd, database = dbName)

***

# Data acquisition and cleaning

## Get Master Dataframe
For now we are using a pickled dataframe until we can get our model sorted out then we will get the data from the  AWS database that we have inputed and can retreive the data needed.

In [6]:
class Stock():
    def __init__(self, symbol):
        self.symbol = symbol
        self.data = []
        self.indicators = []
        
    def getStockData(self, source, user):
        '''This function gets the stock data either from the local computer or online. Returns a dataframe of the 
        one minute stock data.

        Args:
        symbol (string): this is the symbol of the data that is needed.
        source (string): This string is for either online or local hard drive access of stock one minute data.
        user (string): This is the number in the future you want to predict out.

        Returns:
        df (dataframe): stock data by the symbol requested in dataframe sorted by date.  Indicators will be added later.

        Notes: 
        '''
        if source == 'local':
            if user == 'antonio':
                df = pickle.load(open( "D:\Rechin_CCT\Rechin\data\masterDf_Phase4.p", "rb" ))
                df = df[df.Symbol == symbol]
                # Make sure the data is sorted ascending order by datetime.
                df.sort_values(by=['date'], inplace=True)
                df.reset_index(inplace=True, drop=True)
                df.head()
            elif user == 'james':
                df = pickle.load(open( "D:\Downloads\masterDf_Phase4.p", "rb" ))
                df = df[df.Symbol == symbol]
                # Make sure the data is sorted ascending order by datetime.
                df.sort_values(by=['Date'], inplace=True)
                df.reset_index(inplace=True, drop=True)
                df.head()
            else:
                print('User not recognized!')
                df = pd.DataFrame()
        elif source == 'cloud':
            # Credentials to database connection
            hostname="database-rechin.123456.eu-west-1.rds.amazonaws.com" #Antonio's AWS
            uname="admin"
            pwd="rechin123"
            dbName = 'rechindbcct'
            
            # Connect to Specific Database
            db = mysql.connector.connect(host= hostname,
              user=uname, password=pwd, database = dbName)
            
            # Get all symbols from database
            cursor = db.cursor()
            cursor.execute("""SELECT * FROM indicators WHERE symbol = (%s)""", (self.symbol,))
            result = cursor.fetchall()
            
            # Create dataframe for symbol
            columns = [i[0] for i in cursor.description]
            df = pd.DataFrame(result, columns=columns)
        #df = df.drop(columns=['data_vendor','created_date', 'last_updated'])
        df.sort_values(by="Date", inplace=True)
        df.reset_index(inplace=True, drop=True)
        self.data = df
        return df

## Now time to clean our dataset
There are a few things needed to be done here.  We need to drop columns and deal with some 0 values in volume.  From there the dataset is ready to add technical indicators. 

## Now Put our Label into the Last Column of DataFrame
The preceding columns are features and the right column will will be the label column.  This is the column we will tell the machine we want it to predict.

# Data transformations for ML

## Split the DataSet into Train and Test
Based on the split amount we selected above we will split the dataset into two, Train and Test.  Decided to split the dataset this way rather than using the sklearn split function as it shuffles the dataset and we cannot do that in a time series.

## Scale the Dataset for the Machine to Predict
Scaling makes it easier for the machine to predict as it deals with outliers and puts all within a -1 to 1 range.

## Methods to Reshape the dataset and Later Model Evaluation and Predictions
These methods will be used to put into an array the lookback periods into each row removing the first lookback rows.  Also it deletes the prediction rows at the end of the prediction data. It will also turn the data into arrays for us in the model.

In [7]:
class dataTransformer():
    def __init__(self, scaler, data):
        self.scaler = scaler
        self.data = data
    ## Now time to clean our dataset
    #There are a few things needed to be done here.  
    #We need to drop columns and deal with some 0 values in volume.  
    #From there the dataset is ready to add technical indicators.
    def CleanImpute(self, data, impute):
        '''This function replaces missing volume values in the dataset with the rolling volume average.  
        If imputed is true the missing volume values is replaced volume average.  Columns are dropped 
        that are not needed and index is reset.
        If impute is false the column not needed is dropped and the index is reset.

        Args:
        data (dataFrame): This is the data that will be cleaned with imputed data.

        Returns:
        df (dataframe): Cleaned dataframe with either imputed data and removed columns or just removed colomns
        depending if imputed is true or false.

        Notes: 
        '''
        df = data.copy()
        if impute:
            # Clean dirty data
            ### Imputing 20K zeroes with rolling average for the last 360 minutes
            data["volume_mean"]=data["volume"].replace(0, np.nan)
            data["volume_mean"] = data["volume_mean"].fillna(data["volume_mean"].rolling(360,min_periods=1).mean())

            # Now that dataset has imputed values we make our final dataframe to use for our model.
            df = data.copy()
            
            # replace column with missing values with imputed column.
            df['volume'] = df['volume_mean']

            # Now drop columns that will not be needed for our model.
            df = df.drop(columns=['volume_mean'])
            df = df.drop(columns=['Symbol'])
            df.reset_index(inplace=True, drop=True)
        else:
            df = df.drop(columns=['Symbol'])
            df.reset_index(inplace=True, drop=True)

        return df

    def Index_ReorderIndependent(self, df):
        '''This function puts the label into the last column of the dataframe.  
        The preceding columns are features and the right column will will be the label column.  
        This is the column we will tell the machine we want it to predict.

        Args:
        data (dataFrame): This is the data that will be reorderd into label last column.

        Returns:
        df (dataframe): Reordered dataframe with features on rows 0 to second to last and last row is the
        label or target value.

        Notes: 
        We need to add to this function which is the label and do this programatically.  
        '''
        df.set_index(pd.DatetimeIndex(df["Date"]), inplace=True)
        df.drop(columns=['Date'],inplace=True)
        first_column = df.pop('Close')
        # insert column using insert(position,column_name, first_column) function
        df.insert(len(df.columns), 'Close', first_column)
        return df
    
    def GetTrainTest(self, df, train_split_percentage):
        '''This function takes our dataset and splits it into train dataset and test dataset.  This will 
        help our machine learn and then test the model to validate the model.

        Args:
        data (dataFrame): This is the data that you will split into train and test.
        train_split_percentage (float): This is percentage you want to split the dataset by.

        Returns:
        train (array): dataset to use for the training of the model.
        test (array): dataset to validate the model training.

        Notes: 
        '''
        # Get the index where we will split the dataset
        splitIndex = int(round(len(df) * train_split_percentage,0))

        # Split the dataset into two
        train = df.iloc[:splitIndex,:]
        test = df.iloc[splitIndex+1:,:]
        return train, test
    
    def scaleData(self, train, test):
        '''This function scales the dataset to make easier for the LSTM model to train and predict.  This deals
        with outliers.

        Args:
        train (array): dataset to scale for the training of the model.
        test (array): dataset to scale for the testing the model.

        Returns:
        train (array): scaled dataset to use for the training of the model.
        test (array): scaled dataset to validate the model training.
        scaler (scaler model): this model will be used later to inverse transform the predicted label.

        Notes: 
        will add MinMaxScaler to try another scaler and see if this improves accuracy and avoids overfitting.
        '''
        if self.scaler == 'StandardScaler':
            # Call the scaler function
            scaler = StandardScaler()
            # Scale the Train & Test Data with same fitting (scaling factor)
            scaler = scaler.fit(train)
            scaled_train = scaler.transform(train)
            scaled_test = scaler.transform(test)
        elif self.scaler == 'MinMaxScaler':
            # Need to put in minmaxscaler
            pass
        return scaled_train, scaled_test, scaler
    
    # Google style documentation on functions:
    #https://google.github.io/styleguide/pyguide.html
    def reformatDataset(self, n_past, n_future, data):
        '''This function takes the dataset input and returns the data in a format usable by the LSTM Model.  
        The dataset is reshaped to the number of past minutes for each iteration and then 
        gets the predicted minutes.  At the end of the function the data is converted into an array.

        Args:
        n_past (int): This is the number of past observations to go back to use in each row for the LSTM.
        n_future (int): This is the number in the future you want to predict out.
        data (dataFrame): This is the data that you will reshape into features and label.

        Returns:
        features (array): features in shape ready for LSTM training or LSTM predictions.
        label (array): label ready for LSTM training or LSTM predictions.

        Notes: 
        https://github.com/bnsreenu/python_for_microscopists/blob/master/181_multivariate_timeseries_LSTM_GE.py
        '''
        # Create empty arrays to input the formated data
        xDataset = []
        yDataset = []
        # Loop through the dataset and append both the feature and label array in preparation for LSTM model.
        for i in range(n_past, len(data) - n_future + 1):
            xDataset.append(data[i - n_past:i, 0:data.shape[1]])
            yDataset.append(data[i + n_future - 1:i + n_future, 0])

        # Convert the data into numpy arrays.
        xDataset, yDataset = np.array(xDataset), np.array(yDataset)
        return xDataset, yDataset

# Now Create our RNN LSTM Model and Print Summary
This is model that we have created over various, books, online articles, papers and youtube videos... countless hours.

## Notes on Early Stopping



* **EarlyStopping**: Stop training when a monitored mertric has stopped omproving.

* **<font color=red>monitor</font>**: quantity to be monitored

* **<font color=red>min_delta</font>**: minimum change in the monitored quantity to qualify as an improvement, i.e. an absolute change of less than min_delta will count as no improvement.

* **<font color=red>patience</font>**: numbers of epochs with no improvement after which training will be stopped.

* **ReduceLROnPLateau**: Reduce learning rate when a metric has stopped improving.
* **<font color=red>factor</font>**: factor by which the learning rate will be reduced. <font color=red>new_lr = lr * factor</font>

### Notes on size of n_units

n_neurons = round(self.X_train.shape[1] / (neuron_scale_factor*(self.X_train.shape[2]+1)))


$
N_{h} = \frac{N_{s}}{(\alpha*(N_{i}+N_{o}))}
$

Ni = number of input neurons.

No = number of output neurons.

Ns = number of samples in training data set.

α = an arbitrary scaling factor usually 2-10.

Source: https://ai.stackexchange.com/questions/3156/how-to-select-number-of-hidden-layers-and-number-of-memory-cells-in-an-lstm


In [8]:
class Trainer:
    def __init__(self, loss, optimizer, epochs, X_train, y_train, X_test, y_test, activation, 
            layerNumber, modelName, patience, customBatchSize, neuron_scale_factor, split):
        self.modelName = modelName
        self.layerNumber = layerNumber
        self.model = Sequential()
        self.loss = loss
        self.optimizer = optimizer
        self.epochs = epochs
        self.X_train = X_train
        self.y_train = y_train
        self.activation = activation
        self.patience = patience
        self.customBatchSize = customBatchSize
        self.logs_save_path = "logs/fit/"
        self.model_save_path = "logs/model/"
        self.neuron_scale_factor = neuron_scale_factor
        self.split = split
        
    def createModel(self):
        '''This function creates our model based on a number of input arguements.  Firstly we want to clear
        any prior sessions to get a good clean model.  Number of Nuerons is also processed here based on the
        size of the dataset and features.  Then build the number of layers based on the inputs.  Finally,
        compile the model and print a summary of the model.

        Args:
        loss (string): This string is the model which loss function used in the model's trainging and test.
        optimizer (string): This string is the optimization function used in the model's trainging and test.
        epochs (int): This is int is the number of epochs to use in traning the model.
        X_train (array): This data is the feature data will be used to train the model.
        y_train (array): This data is the label data will be used to train the model.
        X_test (array): This data is the feature data will be used to test the model.
        y_test (array): This data is the label data will be used to test the model.
        activation (string): This is the function that is required between matrix.
        layerNumber (int): How many layers to put into the model.
        modelName (string): The name of the model to save the model and loss functions.
        patience (int): The max number of times the epochs can go without improvement in the loss function.
        customBatchSize (int): Batch sizes of the data to use in the model.

        Returns:
        model (array): features in shape ready for LSTM training or LSTM predictions.
        n_nuerons (array): label ready for LSTM training or LSTM predictions.

        Notes: 

        '''
        #####################################
        #Reset the backend for the next iteration (rerun the model)
        tf.keras.backend.clear_session()
        #####################################

        # Initialize the model
        model = self.model

        # Model with n_neurons = inputshape Timestamps, each with x_train.shape[2] variables
        #n_neurons = self.X_train.shape[1] * self.X_train.shape[2]
    
        n_neurons = round(self.X_train.shape[1] / (self.neuron_scale_factor*(self.X_train.shape[2]+1)))
        
        layerNumber = self.layerNumber+3
        
        print('Number of look back minutes: {}\nNumber of features: {}\nTrainTest split: {}\nBatches: {}\nNumber of Neurons: {}\nLayers: {}'.
              format(self.X_train.shape[1], self.X_train.shape[2], self.split, self.customBatchSize, n_neurons, layerNumber))

        # These two layers work but not sure if it is a proper LSTM
        model.add(LSTM(units=n_neurons, activation='tanh', input_shape=(X_train.shape[1], X_train.shape[2]),
                       return_sequences=True, name=self.modelName + "_Input"))

        if self.layerNumber > 1:
            for layer in range(self.layerNumber):
                    self.layerName = self.modelName+"_"+str(layer)
                    if layer < self.layerNumber:
                        model.add(LSTM(units=n_neurons, activation=self.activation, 
                                       return_sequences=True, name =  self.layerName))
                    elif layer == self.layerNumber:
                        self.layerName = self.modelName+"_"+str(layer)
                        model.add(LSTM(units=n_neurons, activation=self.activation, 
                                       return_sequences=False, name =  self.layerName))

        model.add(Dropout(0.20))
        model.add(Dense(y_train.shape[1]))

        model.compile(optimizer=self.optimizer, loss=self.loss,
                      metrics=[metrics_tf.mean_squared_error,
                               metrics_tf.mean_absolute_error, 
                               metrics_tf.mean_absolute_percentage_error])

        model.summary()
        plot_model(model, show_shapes=False)
        return model, n_neurons
    
    def trainModel(self):
        '''This function takes model and add additional callbacks into the model.  Here the model is also
        saved along with the logs of the model during the training process. The model is saved and if a better
        model comes along it is saved and overwrites the model.  This only happens on each model and does not
        overwrite other models.  Patience is added to the callbacks to stop the training early if no improvement.
        Validation data is added to the model to validate the trained model against test data.

        Args:
        They are brought in when the class is called an brought into this function.

        Returns:
        history (model): returns a model after training through the number of epochs.

        Notes: 
        '''
        es = EarlyStopping(monitor='val_loss', min_delta=1e-10, patience=self.patience, verbose=1)
        rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=self.patience, verbose=1)
        #https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint
        modelName = self.modelName +'.h5'
        saveName = os.path.join(self.model_save_path, modelName)
        mcp = ModelCheckpoint(filepath=saveName, monitor='val_loss', verbose=1, save_best_only=True)
        
        # Save logs of callbacks
        logdir=os.path.join(self.logs_save_path, (datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+self.modelName))
        tb = tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1)
        
        # fit the model
        history = self.model.fit(self.X_train, self.y_train, epochs=self.epochs, batch_size=self.customBatchSize, 
                            validation_split=0.1, verbose=1, callbacks=[es, rlr, mcp, tb], 
                            validation_data=(X_test, y_test))

        return history

## User Variables

Here is the inital variables that are needed later in selecting our stock to use, spliting the data and the discrete time inputs.

In [9]:
# Stock symbol we will be using to train our model
symbol = "MNST"
# Past look back minutes to train and amount of futrue minutes to predict
lookback_minutes = 360
prediction_minutes = 10 # AKA timesteps

## LSTM Model parameters

In [10]:
# Split amount for train and test
#train_split " : [0.50, 0.70, 0.90]
#ambitious train_split = [0.50, 0.60, 0.70, 0.80, 0.90]

#50
LSTM_1_dict ={

"train_split" : 0.50, 
"custom_batch_size" : 64, 
"custom_epochs" : 30,
"LSTMLayers" : 3,
"neuronScaleFactor" : 2,
"custom_patience" : 10
}

LSTM_2_dict ={

"train_split" : 0.50, 
"custom_batch_size" : 128, 
"custom_epochs" : 30,
"LSTMLayers" : 3,
"neuronScaleFactor" : 2,
"custom_patience" : 10
}

LSTM_3_dict ={

"train_split" : 0.50, 
"custom_batch_size" : 256, 
"custom_epochs" : 30,
"LSTMLayers" : 3,
"neuronScaleFactor" : 2,
"custom_patience" : 10
}
#70
LSTM_4_dict ={

"train_split" : 0.70, 
"custom_batch_size" : 64, 
"custom_epochs" : 30,
"LSTMLayers" : 3,
"neuronScaleFactor" : 2,
"custom_patience" : 10
}

LSTM_5_dict ={

"train_split" : 0.70, 
"custom_batch_size" : 128, 
"custom_epochs" : 30,
"LSTMLayers" : 3,
"neuronScaleFactor" : 2,
"custom_patience" : 10
}

LSTM_6_dict ={

"train_split" : 0.70, 
"custom_batch_size" : 256, 
"custom_epochs" : 30,
"LSTMLayers" : 3,
"neuronScaleFactor" : 2,
"custom_patience" : 10
}

#90
LSTM_7_dict ={

"train_split" : 0.90, 
"custom_batch_size" : 64, 
"custom_epochs" : 30,
"LSTMLayers" : 3,
"neuronScaleFactor" : 2,
"custom_patience" : 10
}

LSTM_8_dict ={

"train_split" : 0.90, 
"custom_batch_size" : 128, 
"custom_epochs" : 30,
"LSTMLayers" : 3,
"neuronScaleFactor" : 2,
"custom_patience" : 10
}

LSTM_9_dict ={

"train_split" : 0.90, 
"custom_batch_size" : 256, 
"custom_epochs" : 30,
"LSTMLayers" : 3,
"neuronScaleFactor" : 2,
"custom_patience" : 10
}

#70_4_Layers
LSTM_10_dict ={

"train_split" : 0.70, 
"custom_batch_size" : 64, 
"custom_epochs" : 30,
"LSTMLayers" : 4,
"neuronScaleFactor" : 2,
"custom_patience" : 10
}

LSTM_11_dict ={

"train_split" : 0.70, 
"custom_batch_size" : 128, 
"custom_epochs" : 30,
"LSTMLayers" : 4,
"neuronScaleFactor" : 2,
"custom_patience" : 10
}

LSTM_12_dict ={

"train_split" : 0.70, 
"custom_batch_size" : 256, 
"custom_epochs" : 30,
"LSTMLayers" : 4,
"neuronScaleFactor" : 2,
"custom_patience" : 10
}

#70_5_Layers
LSTM_13_dict ={

"train_split" : 0.70, 
"custom_batch_size" : 64, 
"custom_epochs" : 30,
"LSTMLayers" : 5,
"neuronScaleFactor" : 2,
"custom_patience" : 10
}

LSTM_14_dict ={

"train_split" : 0.70, 
"custom_batch_size" : 128, 
"custom_epochs" : 30,
"LSTMLayers" : 5,
"neuronScaleFactor" : 2,
"custom_patience" : 10
}

LSTM_15_dict ={

"train_split" : 0.70, 
"custom_batch_size" : 256, 
"custom_epochs" : 30,
"LSTMLayers" : 5,
"neuronScaleFactor" : 2,
"custom_patience" : 10
}

LSTM_dict_list = [LSTM_1_dict,LSTM_2_dict,LSTM_3_dict,LSTM_4_dict,LSTM_5_dict,LSTM_6_dict,
                  LSTM_7_dict,LSTM_8_dict,LSTM_9_dict,LSTM_10_dict,LSTM_11_dict,LSTM_12_dict,
                  LSTM_13_dict, LSTM_14_dict, LSTM_15_dict ]

***

 ## Function Calling

In [11]:
df = Stock(symbol=symbol).getStockData(source='local',user='james')
df

,Symbol,Date,Open,High,Low,Close,Volume,R3,R2,R1,...,S2,S3,VWAP,CFI,WOBV,OBV,KC_UPPER,KC_LOWER,kijun,senkou
0,MNST,2020-10-15 09:30:00,81.83,82.495,81.740,81.900,1701.0,83.863333,83.246667,82.033333,...,79.586667,78.373333,90.161408,19445.429659,20785.085,22077.0,0,0,81.6475,81.0700
1,MNST,2020-10-15 09:32:00,81.71,81.720,81.710,81.720,105.0,83.863333,83.246667,82.033333,...,79.586667,78.373333,90.161367,19863.457860,20766.185,21972.0,0,0,81.6475,81.0700
2,MNST,2020-10-15 09:33:00,81.99,82.005,81.990,82.005,15.0,83.863333,83.246667,82.033333,...,79.586667,78.373333,90.161361,20016.599152,20770.460,21987.0,0,1,81.6475,81.0700
3,MNST,2020-10-15 09:34:00,81.88,81.950,81.870,81.950,300.0,83.863333,83.246667,82.033333,...,79.586667,78.373333,90.161248,20138.896573,20753.960,21687.0,0,1,81.6475,81.0700
4,MNST,2020-10-15 09:35:00,81.99,81.990,81.745,81.745,45.0,83.863333,83.246667,82.033333,...,79.586667,78.373333,90.161231,20237.280827,20744.735,21642.0,0,1,81.6475,81.0700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77378,MNST,2021-10-14 15:55:00,85.50,85.555,85.480,85.495,3772.0,87.430000,86.940000,86.250000,...,84.580000,83.890000,85.851734,-310.637706,-9.605,-20005.0,0,0,85.4125,85.6050
77379,MNST,2021-10-14 15:56:00,85.51,85.530,85.435,85.435,1398.0,87.430000,86.940000,86.250000,...,84.580000,83.890000,85.848557,-282.923584,-93.485,-21403.0,0,1,85.4125,85.6050
77380,MNST,2021-10-14 15:57:00,85.47,85.520,85.450,85.520,2563.0,87.430000,86.940000,86.250000,...,84.580000,83.890000,85.843315,-220.638392,124.370,-18840.0,0,1,85.4125,85.5950
77381,MNST,2021-10-14 15:58:00,85.50,85.515,85.480,85.500,2803.0,87.430000,86.940000,86.250000,...,84.580000,83.890000,85.837784,-179.870509,68.310,-21643.0,0,1,85.4125,85.5950


In [12]:
transform = dataTransformer(scaler='StandardScaler', data=df)

In [13]:
df = transform.CleanImpute(data=df, impute=False)
df

,Date,Open,High,Low,Close,Volume,R3,R2,R1,PP,...,S2,S3,VWAP,CFI,WOBV,OBV,KC_UPPER,KC_LOWER,kijun,senkou
0,2020-10-15 09:30:00,81.83,82.495,81.740,81.900,1701.0,83.863333,83.246667,82.033333,81.416667,...,79.586667,78.373333,90.161408,19445.429659,20785.085,22077.0,0,0,81.6475,81.0700
1,2020-10-15 09:32:00,81.71,81.720,81.710,81.720,105.0,83.863333,83.246667,82.033333,81.416667,...,79.586667,78.373333,90.161367,19863.457860,20766.185,21972.0,0,0,81.6475,81.0700
2,2020-10-15 09:33:00,81.99,82.005,81.990,82.005,15.0,83.863333,83.246667,82.033333,81.416667,...,79.586667,78.373333,90.161361,20016.599152,20770.460,21987.0,0,1,81.6475,81.0700
3,2020-10-15 09:34:00,81.88,81.950,81.870,81.950,300.0,83.863333,83.246667,82.033333,81.416667,...,79.586667,78.373333,90.161248,20138.896573,20753.960,21687.0,0,1,81.6475,81.0700
4,2020-10-15 09:35:00,81.99,81.990,81.745,81.745,45.0,83.863333,83.246667,82.033333,81.416667,...,79.586667,78.373333,90.161231,20237.280827,20744.735,21642.0,0,1,81.6475,81.0700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77378,2021-10-14 15:55:00,85.50,85.555,85.480,85.495,3772.0,87.430000,86.940000,86.250000,85.760000,...,84.580000,83.890000,85.851734,-310.637706,-9.605,-20005.0,0,0,85.4125,85.6050
77379,2021-10-14 15:56:00,85.51,85.530,85.435,85.435,1398.0,87.430000,86.940000,86.250000,85.760000,...,84.580000,83.890000,85.848557,-282.923584,-93.485,-21403.0,0,1,85.4125,85.6050
77380,2021-10-14 15:57:00,85.47,85.520,85.450,85.520,2563.0,87.430000,86.940000,86.250000,85.760000,...,84.580000,83.890000,85.843315,-220.638392,124.370,-18840.0,0,1,85.4125,85.5950
77381,2021-10-14 15:58:00,85.50,85.515,85.480,85.500,2803.0,87.430000,86.940000,86.250000,85.760000,...,84.580000,83.890000,85.837784,-179.870509,68.310,-21643.0,0,1,85.4125,85.5950


In [14]:
df = transform.Index_ReorderIndependent(df)
df

,Open,High,Low,Volume,R3,R2,R1,PP,S1,S2,S3,VWAP,CFI,WOBV,OBV,KC_UPPER,KC_LOWER,kijun,senkou,Close
Date,,,,,,,,,,,,,,,,,,,,
2020-10-15 09:30:00,81.83,82.495,81.740,1701.0,83.863333,83.246667,82.033333,81.416667,80.203333,79.586667,78.373333,90.161408,19445.429659,20785.085,22077.0,0,0,81.6475,81.0700,81.900
2020-10-15 09:32:00,81.71,81.720,81.710,105.0,83.863333,83.246667,82.033333,81.416667,80.203333,79.586667,78.373333,90.161367,19863.457860,20766.185,21972.0,0,0,81.6475,81.0700,81.720
2020-10-15 09:33:00,81.99,82.005,81.990,15.0,83.863333,83.246667,82.033333,81.416667,80.203333,79.586667,78.373333,90.161361,20016.599152,20770.460,21987.0,0,1,81.6475,81.0700,82.005
2020-10-15 09:34:00,81.88,81.950,81.870,300.0,83.863333,83.246667,82.033333,81.416667,80.203333,79.586667,78.373333,90.161248,20138.896573,20753.960,21687.0,0,1,81.6475,81.0700,81.950
2020-10-15 09:35:00,81.99,81.990,81.745,45.0,83.863333,83.246667,82.033333,81.416667,80.203333,79.586667,78.373333,90.161231,20237.280827,20744.735,21642.0,0,1,81.6475,81.0700,81.745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-14 15:55:00,85.50,85.555,85.480,3772.0,87.430000,86.940000,86.250000,85.760000,85.070000,84.580000,83.890000,85.851734,-310.637706,-9.605,-20005.0,0,0,85.4125,85.6050,85.495
2021-10-14 15:56:00,85.51,85.530,85.435,1398.0,87.430000,86.940000,86.250000,85.760000,85.070000,84.580000,83.890000,85.848557,-282.923584,-93.485,-21403.0,0,1,85.4125,85.6050,85.435
2021-10-14 15:57:00,85.47,85.520,85.450,2563.0,87.430000,86.940000,86.250000,85.760000,85.070000,84.580000,83.890000,85.843315,-220.638392,124.370,-18840.0,0,1,85.4125,85.5950,85.520


In [15]:
n_features = len(df.columns)

## Model Fitting

Train each model 3 times in a loop and save best model.

In [ ]:
%%time
# Loop different models
for LSTM_dict in LSTM_dict_list:
    # Split the data based on split size
    train, test = transform.GetTrainTest(df, LSTM_dict["train_split"])
    # Scale the data
    scaled_train, scaled_test, scaler = transform.scaleData(train, test)
    
    # Transform our dataset into features and label variable and reshaped 3D 
    # First the Train Dataset
    X_train, y_train = transform.reformatDataset(lookback_minutes, prediction_minutes, scaled_train)
    # Second the Test Dataset
    X_test, y_test = transform.reformatDataset(lookback_minutes, prediction_minutes, scaled_test)
    
    #modelName = 'LSTM_model_layers_' + str(LSTMLayers+2) + '_train_split_' + str(split)
    modelName = 'LSTM_' + str(LSTM_dict["LSTMLayers"]+3) + '_layers_tS_' + str(LSTM_dict["train_split"]) + '_bs_'+str(LSTM_dict["custom_batch_size"])
    
    # Create the model
    model1 = Trainer(loss ='mse', optimizer='adam', epochs=LSTM_dict["custom_epochs"], X_train=X_train, y_train=y_train, 
                 X_test=X_test, y_test=y_test, activation='tanh', layerNumber=LSTM_dict["LSTMLayers"], modelName=modelName, 
                 patience=LSTM_dict["custom_patience"], customBatchSize=LSTM_dict["custom_batch_size"], 
                    neuron_scale_factor = LSTM_dict["neuronScaleFactor"], split = LSTM_dict["train_split"])
    model, n_neurons = model1.createModel()
    
    for i in range(3):
        #Fit the model
        history = model1.trainModel()
    history.model.save('./logs/model/' + modelName + '_Train_3Times.h5')

    
    # Model Evaluation
    evaluation = pd.DataFrame(columns = ['Model', 'Test Size', 'MAE', 'MSE', 'Model Details'])
    evaluation.loc[0] = ['LSTM_Train_3times', str(LSTM_dict["train_split"]), min(history.history['mean_absolute_error']), 
                         min(history.history['mean_squared_error']), modelName]
    # Write data to database
    engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=dbName, user=uname, pw=pwd))
    evaluation.to_sql(name='model_evaluation', con=engine, if_exists='append', index=False)
    

## Model Fitting

Train models once and see which does better.

In [17]:
%%time
# Loop different models
for LSTM_dict in LSTM_dict_list:
    # Split the data based on split size
    train, test = transform.GetTrainTest(df, LSTM_dict["train_split"])
    # Scale the data
    scaled_train, scaled_test, scaler = transform.scaleData(train, test)
    
    # Transform our dataset into features and label variable and reshaped 3D 
    # First the Train Dataset
    X_train, y_train = transform.reformatDataset(lookback_minutes, prediction_minutes, scaled_train)
    # Second the Test Dataset
    X_test, y_test = transform.reformatDataset(lookback_minutes, prediction_minutes, scaled_test)
    
    #modelName = 'LSTM_model_layers_' + str(LSTMLayers+2) + '_train_split_' + str(split)
    modelName = 'LSTM_' + str(LSTM_dict["LSTMLayers"]+3) + '_layers_tS_' + str(LSTM_dict["train_split"]) + '_bs_'+str(LSTM_dict["custom_batch_size"])
    
    # Create the model
    model1 = Trainer(loss ='mse', optimizer='adam', epochs=LSTM_dict["custom_epochs"], X_train=X_train, y_train=y_train, 
                 X_test=X_test, y_test=y_test, activation='tanh', layerNumber=LSTM_dict["LSTMLayers"], modelName=modelName, 
                 patience=LSTM_dict["custom_patience"], customBatchSize=LSTM_dict["custom_batch_size"], 
                    neuron_scale_factor = LSTM_dict["neuronScaleFactor"], split = LSTM_dict["train_split"])
    model, n_neurons = model1.createModel()
    
    history = model1.trainModel()
    history.model.save('./logs/model/' + modelName + '.h5')

    
    # Model Evaluation
    evaluation = pd.DataFrame(columns = ['Model', 'Test Size', 'MAE', 'MSE', 'Model Details'])
    evaluation.loc[0] = ['LSTM', str(LSTM_dict["train_split"]), min(history.history['mean_absolute_error']), 
                         min(history.history['mean_squared_error']), modelName]
    # Write data to database
    engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=dbName, user=uname, pw=pwd))
    evaluation.to_sql(name='model_evaluation', con=engine, if_exists='append', index=False)
    

Number of look back minutes: 360
Number of features: 20
TrainTest split: 0.5
Batches: 64
Number of Neurons: 9
Layers: 6
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
LSTM_6_layers_tS_0.5_bs_64_I (None, 360, 9)            1080      
_________________________________________________________________
LSTM_6_layers_tS_0.5_bs_64_0 (None, 360, 9)            684       
_________________________________________________________________
LSTM_6_layers_tS_0.5_bs_64_1 (None, 360, 9)            684       
_________________________________________________________________
LSTM_6_layers_tS_0.5_bs_64_2 (None, 360, 9)            684       
_________________________________________________________________
dropout (Dropout)            (None, 360, 9)            0         
_________________________________________________________________
dense (Dense)                (None, 360, 1)            10        
To

Epoch 1/30
270/270 [==============================] - 22s 67ms/step - loss: 0.1647 - mean_squared_error: 0.1647 - mean_absolute_error: 0.2755 - mean_absolute_percentage_error: 235.8018 - val_loss: 0.5635 - val_mean_squared_error: 0.5635 - val_mean_absolute_error: 0.6917 - val_mean_absolute_percentage_error: 41.9858

Epoch 00001: val_loss improved from inf to 0.56346, saving model to logs/model\LSTM_6_layers_tS_0.5_bs_128.h5
Epoch 2/30
270/270 [==============================] - 17s 62ms/step - loss: 0.0664 - mean_squared_error: 0.0664 - mean_absolute_error: 0.1836 - mean_absolute_percentage_error: 193.4561 - val_loss: 0.5434 - val_mean_squared_error: 0.5434 - val_mean_absolute_error: 0.6878 - val_mean_absolute_percentage_error: 41.9298

Epoch 00002: val_loss improved from 0.56346 to 0.54341, saving model to logs/model\LSTM_6_layers_tS_0.5_bs_128.h5
Epoch 3/30
270/270 [==============================] - 17s 63ms/step - loss: 0.0603 - mean_squared_error: 0.0603 - mean_absolute_error: 0.176

757/757 [==============================] - 45s 59ms/step - loss: 0.0561 - mean_squared_error: 0.0561 - mean_absolute_error: 0.1717 - mean_absolute_percentage_error: 302.7613 - val_loss: 0.0539 - val_mean_squared_error: 0.0539 - val_mean_absolute_error: 0.1848 - val_mean_absolute_percentage_error: 129.4604

Epoch 00006: val_loss did not improve from 0.02892
Epoch 7/30
757/757 [==============================] - 46s 61ms/step - loss: 0.0550 - mean_squared_error: 0.0550 - mean_absolute_error: 0.1706 - mean_absolute_percentage_error: 305.0388 - val_loss: 0.0584 - val_mean_squared_error: 0.0584 - val_mean_absolute_error: 0.1968 - val_mean_absolute_percentage_error: 130.6238

Epoch 00007: val_loss did not improve from 0.02892
Epoch 8/30
757/757 [==============================] - 47s 62ms/step - loss: 0.0544 - mean_squared_error: 0.0544 - mean_absolute_error: 0.1698 - mean_absolute_percentage_error: 304.6365 - val_loss: 0.0499 - val_mean_squared_error: 0.0499 - val_mean_absolute_error: 0.1812 

379/379 [==============================] - 24s 63ms/step - loss: 0.0546 - mean_squared_error: 0.0546 - mean_absolute_error: 0.1703 - mean_absolute_percentage_error: 292.1621 - val_loss: 0.0162 - val_mean_squared_error: 0.0162 - val_mean_absolute_error: 0.0994 - val_mean_absolute_percentage_error: 63.7900

Epoch 00013: val_loss improved from 0.01769 to 0.01624, saving model to logs/model\LSTM_6_layers_tS_0.7_bs_128.h5
Epoch 14/30
379/379 [==============================] - 24s 63ms/step - loss: 0.0542 - mean_squared_error: 0.0542 - mean_absolute_error: 0.1700 - mean_absolute_percentage_error: 288.6424 - val_loss: 0.0139 - val_mean_squared_error: 0.0139 - val_mean_absolute_error: 0.0901 - val_mean_absolute_percentage_error: 62.3916

Epoch 00014: val_loss improved from 0.01624 to 0.01386, saving model to logs/model\LSTM_6_layers_tS_0.7_bs_128.h5
Epoch 15/30
379/379 [==============================] - 24s 63ms/step - loss: 0.0533 - mean_squared_error: 0.0533 - mean_absolute_error: 0.1687 - m

190/190 [==============================] - 13s 69ms/step - loss: 0.0931 - mean_squared_error: 0.0931 - mean_absolute_error: 0.2123 - mean_absolute_percentage_error: 397.2895 - val_loss: 0.0307 - val_mean_squared_error: 0.0307 - val_mean_absolute_error: 0.1351 - val_mean_absolute_percentage_error: 81.0486

Epoch 00002: val_loss improved from 0.03265 to 0.03071, saving model to logs/model\LSTM_6_layers_tS_0.7_bs_256.h5
Epoch 3/30
190/190 [==============================] - 13s 69ms/step - loss: 0.0817 - mean_squared_error: 0.0817 - mean_absolute_error: 0.1997 - mean_absolute_percentage_error: 368.9072 - val_loss: 0.0275 - val_mean_squared_error: 0.0275 - val_mean_absolute_error: 0.1265 - val_mean_absolute_percentage_error: 79.7788

Epoch 00003: val_loss improved from 0.03071 to 0.02749, saving model to logs/model\LSTM_6_layers_tS_0.7_bs_256.h5
Epoch 4/30
190/190 [==============================] - 13s 71ms/step - loss: 0.0765 - mean_squared_error: 0.0765 - mean_absolute_error: 0.1940 - mea

...

Epoch 1/30
244/244 [==============================] - 42s 134ms/step - loss: 0.1862 - mean_squared_error: 0.1862 - mean_absolute_error: 0.2823 - mean_absolute_percentage_error: 165.9877 - val_loss: 0.1532 - val_mean_squared_error: 0.1532 - val_mean_absolute_error: 0.3375 - val_mean_absolute_percentage_error: 25.1759

Epoch 00001: val_loss improved from inf to 0.15316, saving model to logs/model\LSTM_6_layers_tS_0.9_bs_256.h5
Epoch 2/30
244/244 [==============================] - 29s 118ms/step - loss: 0.0708 - mean_squared_error: 0.0708 - mean_absolute_error: 0.1819 - mean_absolute_percentage_error: 123.6115 - val_loss: 0.1843 - val_mean_squared_error: 0.1843 - val_mean_absolute_error: 0.3927 - val_mean_absolute_percentage_error: 29.7584

Epoch 00002: val_loss did not improve from 0.15316
Epoch 3/30
244/244 [==============================] - 28s 116ms/step - loss: 0.0640 - mean_squared_error: 0.0640 - mean_absolute_error: 0.1735 - mean_absolute_percentage_error: 117.3655 - val_loss: 0.1


Epoch 00002: val_loss did not improve from 0.03491
Epoch 3/30
757/757 [==============================] - 63s 83ms/step - loss: 0.0653 - mean_squared_error: 0.0653 - mean_absolute_error: 0.1820 - mean_absolute_percentage_error: 366.6333 - val_loss: 0.0187 - val_mean_squared_error: 0.0187 - val_mean_absolute_error: 0.1061 - val_mean_absolute_percentage_error: 51.6045

Epoch 00003: val_loss improved from 0.03491 to 0.01866, saving model to logs/model\LSTM_7_layers_tS_0.7_bs_64.h5
Epoch 4/30
757/757 [==============================] - 59s 77ms/step - loss: 0.0614 - mean_squared_error: 0.0614 - mean_absolute_error: 0.1778 - mean_absolute_percentage_error: 368.3101 - val_loss: 0.0273 - val_mean_squared_error: 0.0273 - val_mean_absolute_error: 0.1372 - val_mean_absolute_percentage_error: 66.3204

Epoch 00004: val_loss did not improve from 0.01866
Epoch 5/30
757/757 [==============================] - 60s 80ms/step - loss: 0.0586 - mean_squared_error: 0.0586 - mean_absolute_error: 0.1746 - mean


Epoch 00008: val_loss did not improve from 0.02428
Epoch 9/30
190/190 [==============================] - 16s 86ms/step - loss: 0.0637 - mean_squared_error: 0.0637 - mean_absolute_error: 0.1783 - mean_absolute_percentage_error: 327.9629 - val_loss: 0.0535 - val_mean_squared_error: 0.0535 - val_mean_absolute_error: 0.1658 - val_mean_absolute_percentage_error: 116.2015

Epoch 00009: val_loss did not improve from 0.02428
Epoch 10/30
190/190 [==============================] - 16s 86ms/step - loss: 0.0617 - mean_squared_error: 0.0617 - mean_absolute_error: 0.1762 - mean_absolute_percentage_error: 326.5558 - val_loss: 0.0659 - val_mean_squared_error: 0.0659 - val_mean_absolute_error: 0.2074 - val_mean_absolute_percentage_error: 121.6943

Epoch 00010: val_loss did not improve from 0.02428
Epoch 11/30
190/190 [==============================] - 17s 88ms/step - loss: 0.0607 - mean_squared_error: 0.0607 - mean_absolute_error: 0.1754 - mean_absolute_percentage_error: 318.3307 - val_loss: 0.0749 - 


Epoch 00013: val_loss did not improve from 0.01434
Epoch 14/30
757/757 [==============================] - 77s 101ms/step - loss: 0.0525 - mean_squared_error: 0.0525 - mean_absolute_error: 0.1656 - mean_absolute_percentage_error: 260.8055 - val_loss: 0.0278 - val_mean_squared_error: 0.0278 - val_mean_absolute_error: 0.1324 - val_mean_absolute_percentage_error: 56.5429

Epoch 00014: val_loss did not improve from 0.01434
Epoch 15/30
757/757 [==============================] - 77s 102ms/step - loss: 0.0515 - mean_squared_error: 0.0515 - mean_absolute_error: 0.1647 - mean_absolute_percentage_error: 267.8443 - val_loss: 0.0249 - val_mean_squared_error: 0.0249 - val_mean_absolute_error: 0.1285 - val_mean_absolute_percentage_error: 52.4437

Epoch 00015: val_loss did not improve from 0.01434
Epoch 16/30
757/757 [==============================] - 77s 102ms/step - loss: 0.0506 - mean_squared_error: 0.0506 - mean_absolute_error: 0.1633 - mean_absolute_percentage_error: 258.8280 - val_loss: 0.0352 


Epoch 00011: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 00011: val_loss did not improve from 0.02667
Epoch 00011: early stopping
Wall time: 2h 53min 52s


In [18]:
#!kill 15864

#https://stackoverflow.com/questions/57228487/valueerror-duplicate-plugins-for-name-projector
#https://github.com/tensorflow/tensorboard/issues/2483
#https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks
#https://www.programcreek.com/python/example/104420/keras.callbacks.TensorBoard

%tensorboard --port 6006 --logdir logs/fit
